In [2]:
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

data_dir = "folds_transformed"
root_dir = "folds_transformed_models"

# Helper function to calculate top-3 accuracy
def calculate_top3_accuracy(y_true, y_probs):
    top_3_preds = np.argsort(y_probs, axis=1)[:, -3:]
    matches = [y_true[i] in top_3_preds[i] for i in range(len(y_true))]
    return np.mean(matches)

def calculate_threshold_accuracy(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) <= 1)

def evaluate_metrics(y_true, y_pred, y_probs=None):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'f1_score': f1_score(y_true, y_pred, average='weighted'),
        'classification_report': classification_report(y_true, y_pred, zero_division=0),
        'threshold_accuracy': calculate_threshold_accuracy(y_true, y_pred)
    }
    if y_probs is not None:
        metrics['top_3_accuracy'] = calculate_top3_accuracy(y_true, y_probs)
    return metrics

def load_data(file_path):
    print(f"Loading data from: {file_path}")
    data = pd.read_csv(file_path)
    label_columns = [col for col in data.columns if "slider" in col]
    feature_columns = [col for col in data.columns if col not in label_columns + ["Timestamp", "participant_id"]]
    X = data[feature_columns]
    return X, label_columns, data

def train_model_per_label(X_train, y_train, X_test, y_test, model_save_path, label_name, model_type):
    os.makedirs(model_save_path, exist_ok=True)
    
    model_file = os.path.join(model_save_path, f"{model_type}_{label_name}.{'pkl' if model_type == 'rf' else 'h5'}")
    
    if os.path.exists(model_file):
        print(f"Skipping {model_type.upper()} for {label_name}, already trained.")
        return None
    
    print(f"Training {model_type.upper()} for {label_name}")
    
    if model_type == "rf":
        model = RandomForestClassifier(random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probs = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
        with open(model_file, "wb") as f:
            pickle.dump(model, f)
    else:
        input_shape = (X_train.shape[1], 1)
        X_train, X_test = np.expand_dims(X_train, axis=2), np.expand_dims(X_test, axis=2)
        
        model = Sequential()
        if model_type == "cnn":
            model.add(Conv1D(64, kernel_size=3, activation="relu", input_shape=input_shape))
            model.add(Conv1D(32, kernel_size=3, activation="relu"))
            model.add(Flatten())
        else:  # LSTM
            model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
            model.add(LSTM(32))
        
        model.add(Dense(128, activation="relu"))
        model.add(Dropout(0.3))
        model.add(Dense(11, activation="softmax"))
        
        model.compile(optimizer=Adam(0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
        model.save(model_file)
        y_probs = model.predict(X_test)
        y_pred = y_probs.argmax(axis=1)
    
    return evaluate_metrics(y_test, y_pred, y_probs)

def train_all_models():
    results = []
    folds = sorted(set(f.split("_")[1] for f in os.listdir(data_dir) if f.endswith(".csv")))

    for fold in tqdm(folds, desc="Processing Folds"):
        for data_type in ["pca", "scaled"]:
            train_file = os.path.join(data_dir, f"fold_{fold}_train_{data_type}.csv")
            test_file = os.path.join(data_dir, f"fold_{fold}_test_{data_type}.csv")
            
            if not os.path.exists(train_file) or not os.path.exists(test_file):
                print(f"Skipping fold {fold} ({data_type}): Missing train or test data")
                continue
            
            X_train, label_columns, train_data = load_data(train_file)
            X_test, _, test_data = load_data(test_file)
            
            for label_column in label_columns:
                fold_save_dir = os.path.join(root_dir, f"fold_{fold}", data_type, label_column)
                
                for model_type in ["rf", "cnn", "lstm"]:
                    results_dir = os.path.join(fold_save_dir, model_type)
                    model_results = train_model_per_label(X_train, train_data[label_column], X_test, test_data[label_column], results_dir, label_column, model_type)
                    
                    if model_results:
                        results.append({"Fold": fold, "DataType": data_type, "Label": label_column, "Model": model_type.upper(), **model_results})
    
    results_df = pd.DataFrame(results)
    results_csv_path = os.path.join(root_dir, "fold_model_transformed_performance_results.csv")
    results_df.to_csv(results_csv_path, index=False)
    print(f"Model training results saved to {results_csv_path}")

if __name__ == "__main__":
    train_all_models()


Processing Folds:   0%|          | 0/5 [00:00<?, ?it/s]

Loading data from: folds_transformed/fold_1_train_pca.csv
Loading data from: folds_transformed/fold_1_test_pca.csv
Skipping RF for P1 (sliderNeutralPos), already trained.
Skipping CNN for P1 (sliderNeutralPos), already trained.
Skipping LSTM for P1 (sliderNeutralPos), already trained.
Skipping RF for N1 (sliderNeutralPos), already trained.
Skipping CNN for N1 (sliderNeutralPos), already trained.
Training LSTM for N1 (sliderNeutralPos)
Epoch 1/10
134/134 [==============================] - 7s 37ms/step - loss: 2.1453 - accuracy: 0.2206
Epoch 2/10
134/134 [==============================] - 5s 39ms/step - loss: 2.0972 - accuracy: 0.2246
Epoch 3/10
134/134 [==============================] - 5s 39ms/step - loss: 2.0906 - accuracy: 0.2331
Epoch 4/10
134/134 [==============================] - 5s 41ms/step - loss: 2.0791 - accuracy: 0.2347
Epoch 5/10
134/134 [==============================] - 6s 41ms/step - loss: 2.0769 - accuracy: 0.2302
Epoch 6/10
134/134 [==============================] - 5s

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - 1s 13ms/step
Training RF for P2 (sliderNeutralPos)
Training CNN for P2 (sliderNeutralPos)
Epoch 1/10
134/134 [==============================] - 1s 6ms/step - loss: 1.9633 - accuracy: 0.2771
Epoch 2/10
134/134 [==============================] - 1s 6ms/step - loss: 1.6914 - accuracy: 0.3702
Epoch 3/10
134/134 [==============================] - 1s 6ms/step - loss: 1.5136 - accuracy: 0.4549
Epoch 4/10
134/134 [==============================] - 1s 6ms/step - loss: 1.3314 - accuracy: 0.5182
Epoch 5/10
134/134 [==============================] - 1s 6ms/step - loss: 1.1728 - accuracy: 0.5754
Epoch 6/10
134/134 [==============================] - 1s 6ms/step - loss: 1.0193 - accuracy: 0.6377
Epoch 7/10
134/134 [==============================] - 1s 6ms/step - loss: 0.8704 - accuracy: 0.6891
Epoch 8/10
134/134 [==============================] - 1s 6ms/step - loss: 0.7706 - accuracy: 0.7301
Epoch 9/10
134/134 [==============================] - 1s 6ms/step - l

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

Training LSTM for PRO3 (sliderNeutralPos)
Epoch 1/10
134/134 [==============================] - 19s 126ms/step - loss: 2.1611 - accuracy: 0.1932
Epoch 2/10
134/134 [==============================] - 15s 114ms/step - loss: 2.0865 - accuracy: 0.2011
Epoch 3/10
134/134 [==============================] - 16s 117ms/step - loss: 2.0820 - accuracy: 0.1988
Epoch 4/10
134/134 [==============================] - 16s 122ms/step - loss: 2.0654 - accuracy: 0.2039
Epoch 5/10
134/134 [==============================] - 17s 125ms/step - loss: 2.0575 - accuracy: 0.2110
Epoch 6/10
134/134 [==============================] - 17s 127ms/step - loss: 2.0464 - accuracy: 0.2150
Epoch 7/10
134/134 [==============================] - 17s 124ms/step - loss: 2.0439 - accuracy: 0.2236
Epoch 8/10
134/134 [==============================] - 17s 124ms/step - loss: 2.0305 - accuracy: 0.2253
Epoch 9/10
134/134 [==============================] - 17s 125ms/step - loss: 2.0219 - accuracy: 0.2255
Epoch 10/10
134/134 [==========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - 2s 37ms/step


Processing Folds: 100%|██████████| 5/5 [6:03:59<00:00, 4367.83s/it]  

Model training results saved to folds_transformed_models/fold_model_transformed_performance_results.csv
